In [1]:
from datetime import date
import ast
import pandas as pd

pd.set_option('display.max_columns', None)
df = catalog.load('raw/openaire/researchproduct_collectedfrom_datasource_dev#parquet')

[01/13/25 19:50:25] INFO     Loading data from                                                  ]8;id=486241;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=56401;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             raw/openaire/researchproduct_collectedfrom_datasource_dev#parquet                     
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 49
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author              300 non-null    object
 1   openAccessColor     0 non-null      object
 2   publiclyFunded      0 non-null      object
 3   type                300 non-null    object
 4   language            300 non-null    object
 5   subjects            300 non-null    object
 6   mainTitle           300 non-null    object
 7   description         300 non-null    object
 8   publicationDate     300 non-null    object
 9   publisher           54 non-null     object
 10  format              300 non-null    object
 11  bestAccessRight     300 non-null    object
 12  id                  300 non-null    object
 13  originalId          300 non-null    object
 14  indicators          300 non-null    object
 15  instance            300 non-null    object
 16  isGreen             0 non-null  

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'format', 'bestAccessRight', 'id', 'originalId', 'indicators',
       'instance', 'isGreen', 'isInDiamondJournal', 'contributor', 'country'],
      dtype='object')

In [4]:
df.head(5)

,author,openAccessColor,publiclyFunded,type,language,subjects,mainTitle,description,publicationDate,publisher,format,bestAccessRight,id,originalId,indicators,instance,isGreen,isInDiamondJournal,contributor,country
0,"[{'fullName': 'Miniussi, Claudio L.', 'name': ...",None,None,other,"{'code': 'Español', 'label': 'Español'}","[{'provenance': None, 'subject': {'scheme': 'k...",Aplicacion de la fluorescencia de Rayos X al a...,[La fluorescencia de Rayos X puede ser aplicad...,1969-01-01,Laboratorio de Entrenamiento Multidisciplinari...,"[application/pdf, 11 p.]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",dedup_wf_002::0084e248427bc09d1ea962ee8254febc,"[oai:digital.cic.gba.gob.ar:11746/755, 50|od__...","{'citationImpact': {'citationClass': 'C5', 'ci...","[{'license': None, 'publicationDate': '1969-01...",None,None,None,None
1,"[{'fullName': 'De Giusti, Marisa Raquel', 'nam...",None,None,other,"{'code': 'und', 'label': 'Undetermined'}","[{'provenance': None, 'subject': {'scheme': 'k...",2º Taller para la generación de perfiles de in...,[Objetivos del taller: - Entender la necesida...,2018-10-01,None,"[application/pdf, 55 diap.]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,[50|od______9441::40276da60c9d26730d7d2c5d53f9...,"{'citationImpact': {'citationClass': 'C5', 'ci...","[{'license': None, 'publicationDate': '2018-10...",None,None,None,None
2,"[{'fullName': 'Pesado, Patricia Mabel', 'name'...",None,None,other,"{'code': 'und', 'label': 'Undetermined'}","[{'provenance': None, 'subject': {'scheme': 'k...",Calidad en el desarrollo de Sistemas de Software,[El proyecto de investigación y desarrollo “Si...,2006-06-01,None,"[application/pdf, 4 p.]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",dedup_wf_002::02486530ee27ea6d8fa443ce0bf3156e,[50|od______9441::d0e0daf7569603798aa06f76cdc1...,"{'citationImpact': {'citationClass': 'C5', 'ci...","[{'license': None, 'publicationDate': '2006-06...",None,None,None,None
3,"[{'fullName': 'Almirón, Adriana E.', 'name': '...",None,None,other,"{'code': 'und', 'label': 'Undetermined'}","[{'provenance': None, 'subject': {'scheme': 'k...","First record of Rineloricaria reisi, R. stella...","[Rineloricaria reisi, R. stellata, and R. zain...",2014-01-01,None,"[application/pdf, p. 1528-1530]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",dedup_wf_002::030e478a4a6206520f0c95a165c4582b,[50|od______9441::25284dcd824f3211323f138deb3d...,"{'citationImpact': {'citationClass': 'C5', 'ci...","[{'license': None, 'publicationDate': '2014-01...",None,None,None,None
4,"[{'fullName': 'Rascio, Vicente J. D.', 'name':...",None,None,other,"{'code': 'und', 'label': 'Undetermined'}","[{'provenance': None, 'subject': {'scheme': 'k...",Contribución al estudio del comportamiento de ...,[Formulaciones antiincrustantes fueron estudia...,1971-01-01,Laboratorio de Entrenamiento Multidisciplinari...,"[application/pdf, p. 77-147]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",dedup_wf_002::04d710c7907d0fc982ef0df1528ac550,"[oai:digital.cic.gba.gob.ar:11746/591, 50|od__...","{'citationImpact': {'citationClass': 'C5', 'ci...","[{'license': 'CC BY', 'publicationDate': '1971...",None,None,None,None


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [5]:
def land_openaire_researchproduct_collectedfrom_datasource(df: pd.DataFrame)-> pd.DataFrame:
    df = df.convert_dtypes()
    df_researchproduct = df.loc[:,
            [
                    'author',
                    'openAccessColor',
                    'publiclyFunded',
                    'type',
                    'language',
                    'subjects', 
                    'mainTitle', 
                    # 'description', 
                    'publicationDate', 
                    'publisher',
                    # 'format', 
                    'bestAccessRight', 
                    'id', 
                    'originalId', 
                    'indicators',
                    # 'instance', 
                    'isGreen', 
                    'isInDiamondJournal',
                    # 'contributor',
                    # 'country'
            ]
    ]

    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df_researchproduct['bestAccessRight'].apply(lambda x: x['label'])

    ## indicators
    # Desanidar la columna 'indicators' usando pd.json_normalize()
    df_indicators = pd.json_normalize(df_researchproduct['indicators']).reset_index(drop=True)

    # Asegurar que ambos DataFrames tienen índices alineados
    df_researchproduct.reset_index(drop=True, inplace=True)

    # Concatenar el DataFrame desanidado con el original
    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']), df_indicators], axis=1)
    df_researchproduct.drop(columns=['language'], inplace=True)

    ## originalId
    df_researchproduct2originalId = df_researchproduct.explode('originalId').reset_index(drop=True)
    df_researchproduct2originalId = df_researchproduct2originalId[['id','originalId']]

    ## author
    df_researchproduct2author = df_researchproduct.explode('author').reset_index(drop=True)
    df_researchproduct2author = df_researchproduct2author[['id','author']]
    df_authors = pd.json_normalize(df_researchproduct2author['author']).reset_index(drop=True)
    df_researchproduct2author = pd.concat([df_researchproduct2author.drop(columns=['author']), df_authors], axis=1)

    ## subjects
    df_researchproduct2subject = df_researchproduct.explode('subjects').reset_index(drop=True)
    df_researchproduct2subject = df_researchproduct2subject[['id','subjects']]
    df_subjects = pd.json_normalize(df_researchproduct2subject['subjects']).reset_index(drop=True)
    df_researchproduct2subject = pd.concat([df_researchproduct2subject.drop(columns=['subjects']), df_subjects], axis=1)

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=['author','subjects','bestAccessRight'], inplace=True)

    df_researchproduct['load_datetime'] = date.today()

    return df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject


In [6]:
df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject = land_openaire_researchproduct_collectedfrom_datasource(df)

In [7]:
df_researchproduct.head(5)

,openAccessColor,publiclyFunded,type,mainTitle,publicationDate,publisher,id,originalId,isGreen,isInDiamondJournal,language_code,language_label,bestAccessRight_label,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,load_datetime
0,None,None,other,Aplicacion de la fluorescencia de Rayos X al a...,1969-01-01,Laboratorio de Entrenamiento Multidisciplinari...,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,"[oai:digital.cic.gba.gob.ar:11746/755, 50|od__...",None,None,Español,Español,OPEN,C5,0.0,0.0,C5,1.881580e-09,C5,1.490730e-10,C5,2025-01-13
1,None,None,other,2º Taller para la generación de perfiles de in...,2018-10-01,<NA>,dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,[50|od______9441::40276da60c9d26730d7d2c5d53f9...,None,None,und,Undetermined,OPEN,C5,0.0,0.0,C5,1.881580e-09,C5,1.200281e-09,C5,2025-01-13
2,None,None,other,Calidad en el desarrollo de Sistemas de Software,2006-06-01,<NA>,dedup_wf_002::02486530ee27ea6d8fa443ce0bf3156e,[50|od______9441::d0e0daf7569603798aa06f76cdc1...,None,None,und,Undetermined,OPEN,C5,0.0,0.0,C5,1.881580e-09,C5,3.028341e-10,C5,2025-01-13
3,None,None,other,"First record of Rineloricaria reisi, R. stella...",2014-01-01,<NA>,dedup_wf_002::030e478a4a6206520f0c95a165c4582b,[50|od______9441::25284dcd824f3211323f138deb3d...,None,None,und,Undetermined,OPEN,C5,0.0,0.0,C5,1.881580e-09,C5,7.031715e-10,C5,2025-01-13
4,None,None,other,Contribución al estudio del comportamiento de ...,1971-01-01,Laboratorio de Entrenamiento Multidisciplinari...,dedup_wf_002::04d710c7907d0fc982ef0df1528ac550,"[oai:digital.cic.gba.gob.ar:11746/591, 50|od__...",None,None,und,Undetermined,OPEN,C5,0.0,0.0,C5,1.881580e-09,C5,1.492291e-10,C5,2025-01-13


In [8]:
df_researchproduct2originalId.head(5)

,id,originalId
0,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,oai:digital.cic.gba.gob.ar:11746/755
1,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,50|od______9441::5d783924d67272da77ae41bb1417e9c8
2,dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,50|od______9441::40276da60c9d26730d7d2c5d53f976bd
3,dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,oai:digital.cic.gba.gob.ar:11746/8515
4,dedup_wf_002::02486530ee27ea6d8fa443ce0bf3156e,50|od______9441::d0e0daf7569603798aa06f76cdc18076


In [9]:
df_researchproduct2author.head(5)

,id,fullName,name,pid,rank,surname
0,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,"Miniussi, Claudio L.",Claudio L.,None,1,Miniussi
1,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,"Pérez, Raúl",Raúl,None,2,Pérez
2,dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,"De Giusti, Marisa Raquel",Marisa Raquel,None,1,Giusti
3,dedup_wf_002::00df9d3169981ad636b245c4eb46fd2e,"Vila, María Marta",María Marta,None,2,Vila
4,dedup_wf_002::02486530ee27ea6d8fa443ce0bf3156e,"Pesado, Patricia Mabel",Patricia Mabel,None,1,Pesado


In [10]:
df_researchproduct2subject

,id,provenance,subject.scheme,subject.value
0,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,keyword,antifouling paints
1,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,keyword,fluorescencia de Rayos X
2,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,keyword,X Ray fluorescence
3,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,keyword,Ingeniería de la Construcción y Estructural
4,dedup_wf_002::0084e248427bc09d1ea962ee8254febc,None,keyword,pinturas anti-incrustantes
...,...,...,...,...
1391,od______9441::0316335c0e914aea497516998e8d8ad1,None,keyword,Biopitting
1392,od______9441::0316335c0e914aea497516998e8d8ad1,None,SDG,11. Sustainability
1393,od______9441::0316335c0e914aea497516998e8d8ad1,None,keyword,Biotecnología del Medio Ambiente
1394,od______9441::0316335c0e914aea497516998e8d8ad1,None,keyword,Cultural heritage
